# Kernel Telemetry Analysis Automation

This notebook automates the process of executing malware samples in a controlled VM environment with kernel-level telemetry collection:
1. Restore VM to snapshot 35
2. Start the VM
3. Install kernel driver (install.bat)
4. Start GP driver service (sc start gp-driver)
5. Execute malware sample (already present in VM)
6. Terminate malware process
7. Process telemetry data (rename main.py.dll to main.py)
8. Run Python analysis script
9. Copy CSV results to shared folder

In [158]:
import os
import time
import subprocess
import itertools
import re
import shutil
import datetime

In [159]:
# Create necessary directories
os.system('mkdir -p [new]screenshots')
os.system('mkdir -p [new]kernel-telemetry-results')

0

In [160]:
# VM configuration
vm_name = 'windows11-clone Clone'  # Updated VM name
snapshot_no = 35  # Updated snapshot number

# Command templates
vm_open_cmd = f'vboxmanage startvm "{vm_name}"'
run_cmd = f'vboxmanage guestcontrol "{vm_name}" run --username wineleven --password therealtreasure -- '
run_cmd += '"C:\\Windows\\System32\\cmd.exe" "/c" '
copyfrom_cmd = f'vboxmanage guestcontrol "{vm_name}" --username wineleven --password therealtreasure copyfrom '

# Paths and directories
output_folder = '[new]kernel-telemetry-results'
hafidz_dir = 'C:\\Users\\wineleven\\Documents\\Hafidz\\'
shared_folder_path = "C:\\Users\\wineleven\\Desktop\\Shared"
z_drive = "Z:"

In [161]:
# Utility functions

def tstamp():
    """Get current timestamp in readable format."""
    t = time.time()
    return time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(t)) + ("%.3f" % (t % 1))[1:]

def check_output_t(command):
    """Execute command and return output."""
    ret = subprocess.check_output(command, shell=True)
    return ret

def run_vm_command(command, wait_time=2):
    """Execute command in VM and wait for completion."""
    full_cmd = run_cmd + f'"{command}"'
    print(f"[{tstamp()}] Executing: {command}")
    try:
        result = subprocess.run(full_cmd, shell=True, capture_output=True, text=True, timeout=60)
        if result.returncode != 0:
            print(f"Warning: Command failed with return code {result.returncode}")
            print(f"Error: {result.stderr}")
        time.sleep(wait_time)
        return result
    except subprocess.TimeoutExpired:
        print(f"Command timed out: {command}")
        return None

In [162]:
# VM operations

def restore(snap_no):
    """Restore VM to specified snapshot."""
    print(f"[{tstamp()}] Restoring snapshot {snap_no}")
    check_output_t(f'vboxmanage snapshot "{vm_name}" restore "Snapshot {snap_no}"')
    print(f"[{tstamp()}] Restoring snapshot {snap_no} done")

def shutdown_and_restore(snap_no):
    """Power off VM and restore to snapshot."""
    print(f"[{tstamp()}] Powering off VM")
    try:
        check_output_t(f'vboxmanage controlvm "{vm_name}" poweroff')
        print(f"[{tstamp()}] Powering off VM done")
    except:
        print('VM (probably) already off')
    time.sleep(3)
    restore(snap_no)

def start_vm():
    """Start the VM."""
    print(f"[{tstamp()}] Starting up VM")
    check_output_t(vm_open_cmd)
    print(f"[{tstamp()}] Starting up VM done")

def take_screenshot(filename):
    """Take a screenshot of the VM."""
    print(f"[{tstamp()}] Taking screenshot")
    os.system(f'vboxmanage controlvm "{vm_name}" screenshotpng [new]screenshots/{filename}.png')
    print(f"[{tstamp()}] Taking screenshot done")

In [163]:
# Kernel driver and telemetry operations

def install_kernel_driver():
    """Install kernel driver using install.bat."""
    print(f"[{tstamp()}] Installing kernel driver")
    # Change to Hafidz directory and run install.bat
    install_cmd = f"cd /d {hafidz_dir} && install.bat"
    run_vm_command(install_cmd, wait_time=3)
    print(f"[{tstamp()}] Kernel driver installation completed")

def start_gp_driver():
    """Start the GP driver service."""
    print(f"[{tstamp()}] Starting GP driver service")
    # Change to Hafidz directory and start service
    start_cmd = f"cd /d {hafidz_dir} && sc start gp-driver"
    run_vm_command(start_cmd, wait_time=3)
    print(f"[{tstamp()}] GP driver service started")
    
def process_alert_data(malware_family, malware_name):
    """Process alert data using IOCTL export-csv command."""
    print(f"[{tstamp()}] Processing alert data")
    print(f"Malware Family: {malware_family}")
    print(f"Malware Name: {malware_name}")
    
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    original_alerts_filename = f"alerts_{timestamp}.csv"

    clean_malware_family = malware_family.split('\\')[-1]
    print(f"Clean Malware Family: {clean_malware_family}")
    
    clean_malware_name = malware_name.replace('.exe', '')
    print(f"Clean Malware Name: {clean_malware_name}")
    renamed_alerts_filename = f"alert_{clean_malware_family}_{clean_malware_name}.csv.dll"
    
    # Jalankan perintah export
    alert_cmd = (
        f'cd /d {hafidz_dir} && '
        f'IOCTL.exe export-csv {original_alerts_filename} alerts && '
        f'ren {original_alerts_filename} {renamed_alerts_filename} '
        f'copy {renamed_alerts_filename} {z_drive}'  # <- Tambahkan ini
    )

    run_vm_command(alert_cmd, wait_time=3)

    print(f"[{tstamp()}] Alert data processing completed - saved as {renamed_alerts_filename}")


def process_telemetry_data(malware_family, malware_name):
    """Process telemetry data: rename dll, run python script, convert CSV to DLL, copy DLL files."""
    print(f"[{tstamp()}] Processing telemetry data")
    print(f"Malware Family: {malware_family}")
    print(f"Malware Name: {malware_name}")

    # Nama file target berdasarkan format yang diinginkan
    clean_malware_family = malware_family.split('\\')[-1]
    print(f"Clean Malware Family: {clean_malware_family}")
    
    clean_malware_name = malware_name.replace('.exe', '')
    print(f"Clean Malware Name: {clean_malware_name}")
    new_name = f"registry_{clean_malware_family}_{clean_malware_name}.csv.dll"

    process_cmd = (
        f"cd /d {hafidz_dir} && "
        f"ren main.py.dll main.py && "
        f"python main.py && "
        f"for %f in (registry_data_*.csv) do ren \"%f\" \"{new_name}\" && "
        f"copy {new_name} {z_drive}"
    )

    run_vm_command(process_cmd, wait_time=3)

    print(f"[{tstamp()}] Telemetry data processing completed")


def copy_dll_results():
    """Copy DLL results from VM to host."""
    print(f"[{tstamp()}] Copying DLL results to host")

    # Search for DLL files in the shared folder (Z: drive maps to shared folder)
    dll_search_cmd = f"dir {z_drive}\\*.dll /b"

    try:
        result = run_vm_command(dll_search_cmd)
        if result and result.stdout:
            dll_files = result.stdout.strip().split('\n')

            for dll_file in dll_files:
                dll_file = dll_file.strip()
                if dll_file:  # Skip empty results
                    src_path = f"{z_drive}\\{dll_file}"
                    dst_path = f"{output_folder}/{dll_file}"

                    # Copy file from VM to host
                    copy_cmd = f'{copyfrom_cmd} "{src_path}" "{dst_path}"'
                    os.system(copy_cmd)
                    print(f"[{tstamp()}] Copied {dll_file} to {output_folder}")
    except Exception as e:
        print(f"[{tstamp()}] Error copying DLL files from VM: {e}")

    print(f"[{tstamp()}] DLL results copying from VM to host completed")

    # Copy DLL files from shared folder to final destination
    print(f"[{tstamp()}] Copying DLL results from sharedfolder to [NEW]DatasetHafidz")

    source_folder = "/home/markany-3/Documents/sharedfolder"
    destination_folder = "/home/markany-3/Documents/[NEW]DatasetHafidz"

    try:
        # Ensure destination folder exists
        os.makedirs(destination_folder, exist_ok=True)

        # Loop all DLL files and copy
        for filename in os.listdir(source_folder):
            if filename.lower().endswith(".dll"):
                src_path = os.path.join(source_folder, filename)
                dst_path = os.path.join(destination_folder, filename)

                shutil.copy2(src_path, dst_path)
                print(f"[{tstamp()}] Copied: {filename}")

    except Exception as e:
        print(f"[{tstamp()}] Error copying DLL files: {e}")

    print(f"[{tstamp()}] DLL results copying completed")

    # 🧹 Delete all files in Z: drive (inside VM)
    print(f"[{tstamp()}] Cleaning up Z: drive contents")
    try:
        cleanup_cmd = f'del /q {z_drive}\\*'
        run_vm_command(cleanup_cmd)
        print(f"[{tstamp()}] Z: drive cleanup completed")
    except Exception as e:
        print(f"[{tstamp()}] Error during Z: drive cleanup: {e}")


In [164]:
# Malware operations

def execute_malware(malware_path):
    print(f"[{tstamp()}] Executing malware sample: {malware_path}")

    full_cmd = [
        "vboxmanage", "guestcontrol", vm_name, "run",
        "--username", "wineleven",
        "--password", "therealtreasure",
        "--", "cmd.exe", "/c", "start", "", malware_path
    ]

    # Jalankan di background tanpa blocking
    subprocess.Popen(full_cmd)

    print(f"[{tstamp()}] Malware execution started")

def wait_malware(wait_time):
    """Wait for specified time with progress indicator."""
    print(f"[{tstamp()}] Waiting for {wait_time} seconds for malware execution")
    for i in range(wait_time):
        print('\r[' + '#' * (i+1) + '_' * (wait_time-i-1) + f'] {i+1}/{wait_time}s', end='')
        time.sleep(1)
    print('\r[' + '#' * wait_time + f'] {wait_time}/{wait_time}s - Complete!')

def terminate_malware(malware_name):
    """Terminate the malware process."""
    print(f"[{tstamp()}] Terminating malware: {malware_name}")
    
    # Kill process by executable name
    kill_cmd = f"taskkill /f /im {malware_name}"
    run_vm_command(kill_cmd, wait_time=2)
    
    print(f"[{tstamp()}] Malware termination completed")

In [165]:
def run_kernel_telemetry_analysis(malware_path, malware_name, malware_family, execution_time=60):
    """Run the complete kernel telemetry analysis workflow."""
    print(f"[{tstamp()}] Starting kernel telemetry analysis for {malware_path}")
    print(f"[{tstamp()}] Malware execution time: {execution_time} seconds")
    
    # Initialize timing
    t0 = time.time()
    
    try:
        # Step 1: Restore snapshot and start VM
        restore(snapshot_no)
        start_vm()
        
        # Step 2: Install kernel driver
        install_kernel_driver()
        
        # Step 3: Start GP driver service
        start_gp_driver()
        
        # Step 5: Execute malware (already present in VM)
        execute_malware(malware_path)
        
        # Step 6: Wait for malware execution
        wait_malware(execution_time)
        
        # Step 7: Take screenshot for evidence
        screenshot_name = f"{malware_name}_execution"
        take_screenshot(screenshot_name)
        
        # Step 8: Terminate malware
        terminate_malware(malware_name)
        
        # Step 9: Process alert data
        process_alert_data(malware_family, malware_name)
        
        # Step 10: Process telemetry data
        process_telemetry_data(malware_family, malware_name)
        
        # Step 11: Copy CSV results to host
        copy_dll_results()
        
        # Step 12: Take final screenshot
        final_screenshot = f"{malware_name}_final"
        take_screenshot(final_screenshot)
        
        # Step 13: Shutdown and restore VM
        shutdown_and_restore(snapshot_no)
        
    except Exception as e:
        print(f"Error during analysis: {e}")
        # Emergency cleanup
        try:
            terminate_malware(malware_name)
        except:
            pass
        shutdown_and_restore(snapshot_no)
    
    elapsed = time.time() - t0
    print(f'[{tstamp()}] Kernel telemetry analysis completed in {elapsed:.1f} seconds.\n')

## Run Analysis for a Single Malware Sample

Use this cell to analyze a single malware sample with kernel telemetry. The malware should already be present in the VM.

In [157]:
# Path to malware sample in VM and its executable name
malware_path = r"D:\new\ransomware_high_confidence\avoslocker\0b1f19ba8740b10ed017671aab023228756a6864fb008bf23f3c606189bdcd98.exe"  # Full path in VM
malware_name = "0b1f19ba8740b10ed017671aab023228756a6864fb008bf23f3c606189bdcd98.exe"  # Just the executable name for process killing
malware_family = r"D:\new\ransomware_high_confidence\avoslocker"

# Execution time in seconds (how long to let malware run)
execution_time = 5  # Default: 60 seconds

# Uncomment the line below to run analysis on a single sample
run_kernel_telemetry_analysis(malware_path, malware_name, malware_family, execution_time)

[2025-05-26 12:04:54.692] Starting kernel telemetry analysis for D:\new\ransomware_high_confidence\avoslocker\0b1f19ba8740b10ed017671aab023228756a6864fb008bf23f3c606189bdcd98.exe
[2025-05-26 12:04:54.692] Malware execution time: 5 seconds
[2025-05-26 12:04:54.692] Restoring snapshot 35


0%...10%...20%...30%...40%...50%...60%...70%...80%...90%...100%


[2025-05-26 12:04:56.473] Restoring snapshot 35 done
[2025-05-26 12:04:56.473] Starting up VM
[2025-05-26 12:05:03.185] Starting up VM done
[2025-05-26 12:05:03.185] Installing kernel driver
[2025-05-26 12:05:03.185] Executing: cd /d C:\Users\wineleven\Documents\Hafidz\ && install.bat
[2025-05-26 12:05:07.220] Kernel driver installation completed
[2025-05-26 12:05:07.220] Starting GP driver service
[2025-05-26 12:05:07.220] Executing: cd /d C:\Users\wineleven\Documents\Hafidz\ && sc start gp-driver
[2025-05-26 12:05:10.368] GP driver service started
[2025-05-26 12:05:10.368] Executing malware sample: D:\new\ransomware_high_confidence\avoslocker\0b1f19ba8740b10ed017671aab023228756a6864fb008bf23f3c606189bdcd98.exe
[2025-05-26 12:05:10.369] Malware execution started
[2025-05-26 12:05:10.369] Waiting for 5 seconds for malware execution
[#####] 5/5s - Complete!
[2025-05-26 12:05:15.369] Taking screenshot
[2025-05-26 12:05:15.429] Taking screenshot done
[2025-05-26 12:05:15.429] Terminating 

0%...10%...20%...30%...40%...50%...60%...70%...80%...90%...100%


[2025-05-26 12:05:34.592] Powering off VM done
[2025-05-26 12:05:37.592] Restoring snapshot 35


0%...10%...20%...30%...40%...50%...

[2025-05-26 12:05:39.004] Restoring snapshot 35 done
[2025-05-26 12:05:39.004] Kernel telemetry analysis completed in 44.3 seconds.



60%...70%...80%...90%...100%


## Run Analysis for Multiple Samples

Use this cell to analyze multiple malware samples already present in the VM.

In [ ]:
import os
import time

# List of ransomware family directories
ransomware_directories = [
    "D:\\new\\ransomware_high_confidence\\avaddon",
    "D:\\new\\ransomware_high_confidence\\avoslocker",
    "D:\\new\\ransomware_high_confidence\\babuk",
    "D:\\new\\ransomware_high_confidence\\bianlian",
    "D:\\new\\ransomware_high_confidence\\blackbasta",
    "D:\\new\\ransomware_high_confidence\\blackbyte",
    "D:\\new\\ransomware_high_confidence\\blackcat",
    "D:\\new\\ransomware_high_confidence\\blackmatter",
    "D:\\new\\ransomware_high_confidence\\bluesky",
    "D:\\new\\ransomware_high_confidence\\clop",
    "D:\\new\\ransomware_high_confidence\\conti",
    "D:\\new\\ransomware_high_confidence\\darkside",
    "D:\\new\\ransomware_high_confidence\\dharma",
    "D:\\new\\ransomware_high_confidence\\doppelpaymer",
    "D:\\new\\ransomware_high_confidence\\exorcist",
    "D:\\new\\ransomware_high_confidence\\gandcrab",
    "D:\\new\\ransomware_high_confidence\\hive",
    "D:\\new\\ransomware_high_confidence\\holyghost",
    "D:\\new\\ransomware_high_confidence\\karma",
    "D:\\new\\ransomware_high_confidence\\lockbit",
    "D:\\new\\ransomware_high_confidence\\lorenz",
    "D:\\new\\ransomware_high_confidence\\makop",
    "D:\\new\\ransomware_high_confidence\\maui",
    "D:\\new\\ransomware_high_confidence\\maze",
    "D:\\new\\ransomware_high_confidence\\mountlocker",
    "D:\\new\\ransomware_high_confidence\\nefilim",
    "D:\\new\\ransomware_high_confidence\\netwalker",
    "D:\\new\\ransomware_high_confidence\\nightsky",
    "D:\\new\\ransomware_high_confidence\\phobos",
    "D:\\new\\ransomware_high_confidence\\playcrypt",
    "D:\\new\\ransomware_high_confidence\\pysa",
    "D:\\new\\ransomware_high_confidence\\quantum",
    "D:\\new\\ransomware_high_confidence\\ragnarok",
    "D:\\new\\ransomware_high_confidence\\ransomexx",
    "D:\\new\\ransomware_high_confidence\\revil",
    "D:\\new\\ransomware_high_confidence\\ryuk",
    "D:\\new\\ransomware_high_confidence\\stop",
    "D:\\new\\ransomware_high_confidence\\thanos",
    "D:\\new\\ransomware_high_confidence\\wastedlocker",
    "D:\\new\\ransomware_high_confidence\\zeppelin"
]

def get_all_malware_samples():
    """
    Scan semua direktori ransomware dan dapatkan list semua file .exe
    Returns: List of tuples (full_path, executable_name, family_name)
    """
    all_samples = []
    
    print(f"[{tstamp()}] Scanning malware directories...")
    
    for directory in ransomware_directories:
        family_name = os.path.basename(directory)
        print(f"[{tstamp()}] Scanning {family_name}...")
        
        # Periksa apakah direktori ada di VM (simulasi dengan list files)
        # Dalam implementasi nyata, ini akan menggunakan VM command
        list_cmd = f"dir {directory}\\*.exe /b"
        
        try:
            result = run_vm_command(list_cmd, wait_time=1)
            
            if result and result.stdout.strip():
                exe_files = result.stdout.strip().split('\n')
                
                for exe_file in exe_files:
                    exe_file = exe_file.strip()
                    if exe_file and exe_file.endswith('.exe'):
                        full_path = f"{directory}\\{exe_file}"
                        all_samples.append((full_path, exe_file, family_name))
                        
                print(f"[{tstamp()}] Found {len(exe_files)} samples in {family_name}")
            else:
                print(f"[{tstamp()}] No .exe files found in {family_name}")
                
        except Exception as e:
            print(f"[{tstamp()}] Error scanning {family_name}: {e}")
            continue
    
    print(f"[{tstamp()}] Total samples found: {len(all_samples)}")
    return all_samples

def run_batch_malware_analysis(execution_time=60, max_samples=None, skip_families=None, only_families=None):
    """
    Jalankan analisis batch untuk semua malware samples
    
    Args:
        execution_time (int): Waktu eksekusi per sample dalam detik
        max_samples (int): Maksimal jumlah sample yang dianalisis (None = semua)
        skip_families (list): List family yang di-skip
        only_families (list): Hanya analisis family tertentu
    """
    
    print(f"[{tstamp()}] =" * 80)
    print(f"[{tstamp()}] STARTING BATCH MALWARE ANALYSIS")
    print(f"[{tstamp()}] Execution time per sample: {execution_time} seconds")
    print(f"[{tstamp()}] =" * 80)
    
    # Dapatkan semua samples
    all_samples = get_all_malware_samples()
    
    if not all_samples:
        print(f"[{tstamp()}] No malware samples found! Exiting...")
        return
    
    # Filter berdasarkan parameter
    filtered_samples = []
    
    for sample in all_samples:
        full_path, exe_name, family_name = sample
        
        # Skip families jika ditentukan
        if skip_families and family_name in skip_families:
            continue
            
        # Hanya analisis families tertentu jika ditentukan
        if only_families and family_name not in only_families:
            continue
            
        filtered_samples.append(sample)
    
    # Batasi jumlah sample jika ditentukan
    if max_samples and len(filtered_samples) > max_samples:
        filtered_samples = filtered_samples[:max_samples]
    
    total_samples = len(filtered_samples)
    print(f"[{tstamp()}] Samples to analyze: {total_samples}")
    
    # Estimasi waktu total
    estimated_time = total_samples * (execution_time + 120)  # +120 untuk overhead setup/cleanup
    hours = estimated_time // 3600
    minutes = (estimated_time % 3600) // 60
    print(f"[{tstamp()}] Estimated total time: {hours}h {minutes}m")
    
    # Konfirmasi
    print(f"[{tstamp()}] Press Enter to continue or Ctrl+C to cancel...")
    input()
    
    # Tracking progress
    successful_analyses = 0
    failed_analyses = 0
    analysis_start_time = time.time()
    
    # Log file untuk tracking
    log_file = f"batch_analysis_log_{time.strftime('%Y%m%d_%H%M%S')}.txt"
    
    with open(log_file, 'w') as log:
        log.write(f"Batch Malware Analysis Log\\n")
        log.write(f"Started: {time.strftime('%Y-%m-%d %H:%M:%S')}\\n")
        log.write(f"Total samples: {total_samples}\\n")
        log.write(f"Execution time per sample: {execution_time}s\\n\\n")
        
        for i, (full_path, exe_name, family_name) in enumerate(filtered_samples, 1):
            
            print(f"\\n[{tstamp()}] " + "=" * 60)
            print(f"[{tstamp()}] SAMPLE {i}/{total_samples}")
            print(f"[{tstamp()}] Family: {family_name}")
            print(f"[{tstamp()}] File: {exe_name}")
            print(f"[{tstamp()}] Path: {full_path}")
            print(f"[{tstamp()}] " + "=" * 60)
            
            log.write(f"Sample {i}/{total_samples}: {family_name}/{exe_name}\\n")
            log.flush()
            
            try:
                # Jalankan analisis
                run_kernel_telemetry_analysis(full_path, exe_name, family_name, execution_time)
                
                successful_analyses += 1
                print(f"[{tstamp()}] ✅ Analysis completed successfully!")
                log.write(f"  Status: SUCCESS\\n")
                
            except Exception as e:
                failed_analyses += 1
                print(f"[{tstamp()}] ❌ Analysis failed: {e}")
                log.write(f"  Status: FAILED - {e}\\n")
                
                # Emergency cleanup jika terjadi error
                try:
                    print(f"[{tstamp()}] Attempting emergency cleanup...")
                    shutdown_and_restore(snapshot_no)
                except:
                    pass
            
            # Progress update
            progress = (i / total_samples) * 100
            elapsed = time.time() - analysis_start_time
            avg_time = elapsed / i
            remaining = (total_samples - i) * avg_time
            
            print(f"[{tstamp()}] Progress: {progress:.1f}% ({i}/{total_samples})")
            print(f"[{tstamp()}] Successful: {successful_analyses}, Failed: {failed_analyses}")
            print(f"[{tstamp()}] Time remaining: {remaining/3600:.1f}h")
            
            log.write(f"  Progress: {progress:.1f}%\\n")
            log.write(f"  Time elapsed: {elapsed/3600:.1f}h\\n\\n")
            log.flush()
            
            # Jeda antar sample
            if i < total_samples:
                print(f"[{tstamp()}] Waiting 5 seconds before next sample...")
                time.sleep(5)
    
    # Summary
    total_time = time.time() - analysis_start_time
    print(f"\\n[{tstamp()}] " + "=" * 80)
    print(f"[{tstamp()}] BATCH ANALYSIS COMPLETED!")
    print(f"[{tstamp()}] Total samples processed: {total_samples}")
    print(f"[{tstamp()}] Successful: {successful_analyses}")
    print(f"[{tstamp()}] Failed: {failed_analyses}")
    print(f"[{tstamp()}] Total time: {total_time/3600:.1f} hours")
    print(f"[{tstamp()}] Log saved to: {log_file}")
    print(f"[{tstamp()}] " + "=" * 80)

def quick_test_analysis():
    """
    Test dengan beberapa sample saja untuk memastikan sistem berjalan
    """
    print(f"[{tstamp()}] Running quick test analysis (3 samples, 30s each)...")
    run_batch_malware_analysis(
        execution_time=30,
        max_samples=3,
        only_families=['avaddon', 'avoslocker']  # Test dengan 2 family saja
    )

def run_specific_families(families, execution_time=60):
    """
    Analisis hanya family tertentu
    
    Args:
        families (list): List nama family yang ingin dianalisis
        execution_time (int): Waktu eksekusi per sample
    """
    print(f"[{tstamp()}] Running analysis for specific families: {families}")
    run_batch_malware_analysis(
        execution_time=execution_time,
        only_families=families
    )

def run_all_families_full_analysis(execution_time=60):
    """
    Analisis semua family dengan full execution time
    """
    print(f"[{tstamp()}] Running FULL ANALYSIS for ALL families...")
    print(f"[{tstamp()}] This will take a very long time!")
    run_batch_malware_analysis(execution_time=execution_time)

# ===============================================================================
# CONTOH PENGGUNAAN:
# ===============================================================================

# 1. Test cepat dengan 3 sample saja (30 detik per sample)
# quick_test_analysis()

# 2. Analisis family tertentu saja
# run_specific_families(['avaddon', 'avoslocker', 'babuk'], execution_time=5)

# 3. Analisis semua family dengan waktu eksekusi pendek (testing)
# run_batch_malware_analysis(execution_time=30, max_samples=10)

# 4. Analisis penuh semua family (HATI-HATI: AKAN SANGAT LAMA!)
run_all_families_full_analysis(execution_time=60)

# 5. Skip family tertentu
# run_batch_malware_analysis(
#     execution_time=45, 
#     skip_families=['stop', 'dharma'],  # Skip family yang banyak sample
#     max_samples=50
# )

[2025-05-26 12:08:02.707] Running FULL ANALYSIS for ALL families...
[2025-05-26 12:08:02.707] This will take a very long time!
[2025-05-26 12:08:02.707] =[2025-05-26 12:08:02.707] =[2025-05-26 12:08:02.707] =[2025-05-26 12:08:02.707] =[2025-05-26 12:08:02.707] =[2025-05-26 12:08:02.707] =[2025-05-26 12:08:02.707] =[2025-05-26 12:08:02.707] =[2025-05-26 12:08:02.707] =[2025-05-26 12:08:02.707] =[2025-05-26 12:08:02.707] =[2025-05-26 12:08:02.707] =[2025-05-26 12:08:02.707] =[2025-05-26 12:08:02.707] =[2025-05-26 12:08:02.707] =[2025-05-26 12:08:02.707] =[2025-05-26 12:08:02.707] =[2025-05-26 12:08:02.707] =[2025-05-26 12:08:02.707] =[2025-05-26 12:08:02.707] =[2025-05-26 12:08:02.707] =[2025-05-26 12:08:02.707] =[2025-05-26 12:08:02.707] =[2025-05-26 12:08:02.707] =[2025-05-26 12:08:02.707] =[2025-05-26 12:08:02.707] =[2025-05-26 12:08:02.707] =[2025-05-26 12:08:02.707] =[2025-05-26 12:08:02.707] =[2025-05-26 12:08:02.707] =[2025-05-26 12:08:02.707] =[2025-05-26 12:08:02.707] =[2025-05-

\n[2025-05-26 12:09:26.963] ============================================================
[2025-05-26 12:09:26.963] SAMPLE 1/1408
[2025-05-26 12:09:26.963] Family: D:\new\ransomware_high_confidence\avaddon
[2025-05-26 12:09:26.963] File: 05af0cf40590aef24b28fa04c6b4998b7ab3b7f26e60c507adb84f3d837778f2.exe
[2025-05-26 12:09:26.963] Path: D:\new\ransomware_high_confidence\avaddon\05af0cf40590aef24b28fa04c6b4998b7ab3b7f26e60c507adb84f3d837778f2.exe
[2025-05-26 12:09:26.963] ============================================================
[2025-05-26 12:09:26.964] Starting kernel telemetry analysis for D:\new\ransomware_high_confidence\avaddon\05af0cf40590aef24b28fa04c6b4998b7ab3b7f26e60c507adb84f3d837778f2.exe
[2025-05-26 12:09:26.964] Malware execution time: 60 seconds
[2025-05-26 12:09:26.964] Restoring snapshot 35


0%...10%...20%...30%...40%...50%...60%...70%...80%...90%...100%


[2025-05-26 12:09:28.750] Restoring snapshot 35 done
[2025-05-26 12:09:28.750] Starting up VM
[2025-05-26 12:09:35.478] Starting up VM done
[2025-05-26 12:09:35.478] Installing kernel driver
[2025-05-26 12:09:35.478] Executing: cd /d C:\Users\wineleven\Documents\Hafidz\ && install.bat
[2025-05-26 12:09:40.241] Kernel driver installation completed
[2025-05-26 12:09:40.241] Starting GP driver service
[2025-05-26 12:09:40.241] Executing: cd /d C:\Users\wineleven\Documents\Hafidz\ && sc start gp-driver
[2025-05-26 12:09:43.390] GP driver service started
[2025-05-26 12:09:43.390] Executing malware sample: D:\new\ransomware_high_confidence\avaddon\05af0cf40590aef24b28fa04c6b4998b7ab3b7f26e60c507adb84f3d837778f2.exe
[2025-05-26 12:09:43.390] Malware execution started
[2025-05-26 12:09:43.390] Waiting for 60 seconds for malware execution
[############################################################] 60/60s - Complete!
[2025-05-26 12:10:43.398] Taking screenshot
[2025-05-26 12:10:43.461] Taking

0%...10%...20%...30%...40%...50%...60%...70%...80%...90%...100%


...

[2025-05-26 12:14:02.564] Powering off VM done
[2025-05-26 12:14:05.564] Restoring snapshot 35


0%...10%...20%...30%...40%...50%...60%...70%...80%...90%...100%


[2025-05-26 12:14:06.590] Restoring snapshot 35 done
[2025-05-26 12:14:06.590] Kernel telemetry analysis completed in 139.9 seconds.

[2025-05-26 12:14:06.591] ✅ Analysis completed successfully!
[2025-05-26 12:14:06.591] Progress: 0.1% (2/1408)
[2025-05-26 12:14:06.591] Successful: 2, Failed: 0
[2025-05-26 12:14:06.591] Time remaining: 54.6h
[2025-05-26 12:14:06.591] Waiting 5 seconds before next sample...
\n[2025-05-26 12:14:11.591] ============================================================
[2025-05-26 12:14:11.591] SAMPLE 3/1408
[2025-05-26 12:14:11.591] Family: D:\new\ransomware_high_confidence\avaddon
[2025-05-26 12:14:11.591] File: 0ff4058f709d278ed662719b9627618c48e7a656c59f6bfecda9081c7cbd742b.exe
[2025-05-26 12:14:11.591] Path: D:\new\ransomware_high_confidence\avaddon\0ff4058f709d278ed662719b9627618c48e7a656c59f6bfecda9081c7cbd742b.exe
[2025-05-26 12:14:11.591] ============================================================
[2025-05-26 12:14:11.591] Starting kernel telemetry an

0%...10%...20%...30%...40%...50%...60%...70%...80%...90%...100%


[2025-05-26 12:14:13.296] Restoring snapshot 35 done
[2025-05-26 12:14:13.296] Starting up VM
[2025-05-26 12:14:19.437] Starting up VM done
[2025-05-26 12:14:19.437] Installing kernel driver
[2025-05-26 12:14:19.437] Executing: cd /d C:\Users\wineleven\Documents\Hafidz\ && install.bat
[2025-05-26 12:14:23.809] Kernel driver installation completed
[2025-05-26 12:14:23.809] Starting GP driver service
[2025-05-26 12:14:23.809] Executing: cd /d C:\Users\wineleven\Documents\Hafidz\ && sc start gp-driver
[2025-05-26 12:14:26.965] GP driver service started
[2025-05-26 12:14:26.965] Executing malware sample: D:\new\ransomware_high_confidence\avaddon\0ff4058f709d278ed662719b9627618c48e7a656c59f6bfecda9081c7cbd742b.exe
[2025-05-26 12:14:26.965] Malware execution started
[2025-05-26 12:14:26.965] Waiting for 60 seconds for malware execution
[############################################################] 60/60s - Complete!
[2025-05-26 12:15:26.974] Taking screenshot
[2025-05-26 12:15:27.039] Taking